In [4]:
""" Module for writing artifical vcfs for testing purposes """

' Module for writing artifical vcfs for testing purposes '

In [1]:
import pandas as pd
import os
import VCF
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
def get_random_intervals():
    """ get set of 1000 random intronic / intergenic intervals, with bedtools """
    
    bed_rand = 'bedtools_random_out.csv'
    ! bedtools random -l 1 -n 1000 -g human.hg38.genome > $bed_rand

    bed_inter = 'bedtools_intersect_out.csv'
    ! bedtools intersect -a $bed_rand -b /Users/lincoln.harris/code/potLuck/hg38-plus.gtf > $bed_inter

    bed_unique = 'bed_unique_loci.csv'
    ! grep -F -xvf $bed_inter $bed_rand > $bed_unique
    
    unique = pd.read_csv('bed_unique_loci.csv', sep='\t', names=['chrom', 'start_pos', 'end_pos', 
                                                        'num', 'score', 'strand'])
    pos_strings = []
    
    for idx, row in unique.iterrows():
        chrom_ = row.chrom
        if '_' not in chrom_:
            pos = row.start_pos
            tup = [chrom_, pos]
            pos_strings.append(tup)
        
    os.remove('bedtools_random_out.csv')
    os.remove('bedtools_intersect_out.csv')
    os.remove('bed_unique_loci.csv')
    return(pos_strings)

In [3]:
def add_garbage_lines(df_):
    """ add a bunch of lines that dont correspond to any gene / amino acid sequence
        MANUALLY """
    
    g_info = 'AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=3.00;QD=30.14;SOR=2.303'
    g_format = 'GT:AD:DP:GQ:PL'
    g_20 = '1/1:2,2:13:38:434,38,0'
    
    garbage_positions = [['chr12', 25284777],  # each of these has been evaluated by hand...
                         ['chr12', 24788289],     # intergenic regions, but close to some of the genes
                         ['chr7', 54880599],      #    we care about
                         ['chr7', 55271013], 
                         ['chr2', 29954866], 
                         ['chr2', 29081787],
                         ['chr7', 140944330],
                         ['chr7', 140238466], 
                         ['chr17', 7636991],
                         ['chr2', 41732258]]
    
    random_positions = get_random_intervals()
    
    for elm in random_positions:
        add_chr = elm[0]
        add_pos = elm[1]
        
        df_ = df_.append({'#CHROM': add_chr, 'POS': add_pos, 'ID': '.', 'REF': 'A', 'ALT': 'C', 
                          'QUAL': 60, 'FILTER': '.', 'INFO': g_info, 'FORMAT': g_format, 
                          '20': g_20}, ignore_index=True)
    
    # random shuffle rows
    df_ = df_.sample(frac=1)
    
    return(df_)

In [4]:
def write_vcf(outStr_, chrom, pos, ref_, alt_, v_count, wt_count):
    """ routine for writing VCF files, from an existing dataframe. 
    essentially just adding in this horrible vcf header. """
    
    cwd = os.getcwd()
    
    with open(cwd + '/../cerebra/vcfheader.txt', 'r') as f: # read in artifical header 
        header = f.read()
        df = pd.DataFrame(columns=['#CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', '20'])
        
        chrom_ = 'chr' + str(chrom)
        pos_ = pos
        id_ = '.'
        qual_ = 60.28
        filter_ = '.'
        info_ = 'AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.000;MLEAC=1;MLEAF=0.500;MQ=3.00;QD=30.14;SOR=2.303'
        format_ = 'GT:AD:DP:GQ:PL'
        twenty = '1/1:' + str(v_count) + ',' + str(wt_count) + ':13:38:434,38,0'
        
        
        df = df.append({'#CHROM': chrom_, 'POS': pos_, 'ID': id_, 'REF': ref_, 'ALT': alt_, 
                        'QUAL': qual_, 'FILTER': filter_, 'INFO': info_, 'FORMAT': format_,
                       '20': twenty}, ignore_index=True)
        
        df = add_garbage_lines(df)
        
        output_VCF = outStr_
        with open(output_VCF, 'w') as vcf:
            vcf.write(header)

        df.to_csv(output_VCF, sep="\t", mode='a', index=False)

In [5]:
# write test set
write_vcf('test0_kras_G13C.vcf', 12, 25245348, 'C', 'A', 2, 2)
write_vcf('test1_kras_G13C.vcf', 12, 25245348, 'C', 'A', 0, 10)
write_vcf('test2_kras_G13C.vcf', 12, 25245348, 'C', 'A', 5, 10)

write_vcf('test3_egfr_L858R.vcf', 7, 55191822, 'T', 'G', 2, 2)
write_vcf('test4_egfr_L858R.vcf', 7, 55191822, 'T', 'G', 0, 10)
write_vcf('test5_egfr_L858R.vcf', 7, 55191822, 'T', 'G', 5, 10)

write_vcf('test6_braf_V600E.vcf', 7, 140753336, 'A', 'T', 2, 2)
write_vcf('test7_braf_V600E.vcf', 7, 140753336, 'A', 'T', 0, 10)
write_vcf('test8_braf_V600E.vcf', 7, 140753336, 'A', 'T', 5, 10)

write_vcf('test9_kras_G12C.vcf', 12, 25245351, 'C', 'A', 2, 2)
write_vcf('test10_kras_G12C.vcf', 12, 25245351, 'C', 'A', 0, 10)
write_vcf('test11_kras_G12C.vcf', 12, 25245351, 'C', 'A', 5, 10)

write_vcf('test12_braf_W450L.vcf', 7, 140781659, 'C', 'A', 2, 2)
write_vcf('test13_braf_W450L.vcf', 7, 140781659, 'C', 'A', 0, 10)
write_vcf('test14_braf_W450L.vcf', 7, 140781659, 'C', 'A', 5, 10)

write_vcf('test15_braf_Q257H.vcf', 7, 140801501, 'C', 'A', 2, 2)
write_vcf('test16_braf_Q257H.vcf', 7, 140801501, 'C', 'A', 0, 10)
write_vcf('test17_braf_Q257H.vcf', 7, 140801501, 'C', 'A', 5, 10)

write_vcf('test18_tp53_S241F.vcf', 17, 7674241, 'G', 'A', 2, 2)
write_vcf('test19_tp53_S241F.vcf', 17, 7674241, 'G', 'A', 0, 10)
write_vcf('test20_tp53_S241F.vcf', 17, 7674241, 'G', 'A', 5, 10)

write_vcf('test21_tp53_P72R.vcf', 17, 7676154, 'G', 'C', 2, 2)
write_vcf('test22_tp53_P72R.vcf', 17, 7676154, 'G', 'C', 0, 10)
write_vcf('test23_tp53_P72R.vcf', 17, 7676154, 'G', 'C', 5, 10)

write_vcf('test24_garbage.vcf', 12, 25284777, 'G', 'C', 2, 2) # none of these should return anything
write_vcf('test25_garbage.vcf', 7, 54880599, 'G', 'C', 0, 10)
write_vcf('test26_garbage.vcf', 2, 41732258, 'G', 'C', 5, 10)

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002"; transcript_id "DQ459430";

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002"; transcript_id "DQ459430";

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002"; transcript_id "DQ459430";

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002"; transcript_id "DQ459430";

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002"; transcript_id "DQ459430";

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002"; transcript_id "DQ459430";

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002"; transcript_id "DQ459430";

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002"; transcript_id "DQ459430";

***** WARNING: File /Users/lincoln.harris/code/potLuck/hg38-plus.gtf has inconsistent naming convention for record:
ERCC-00002	ERCC	exon	1	1061	0.000000	+	.	gene_id "ERCC-00002

In [85]:
# validate
VCF.dataframe('test0_kras_G13C.vcf')

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
0,chr7,136853082,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
1,chr12,23421725,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
2,chr3,189333145,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
3,chr1,108859891,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
4,chr4,83474956,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
5,chr7,114096875,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
6,chr17,81745053,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
7,chr7,80005688,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
8,chr3,27102008,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
9,chr1,164687757,.,A,C,60.0,.,AC=2;AF=1.00;AN=2;DP=7;ExcessHet=3.0103;FS=0.0...
